In [11]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [12]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory+automation' # topic of the search
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [13]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

       For the problem you found, please only output the following: 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 250-300 words)
        - "User group": Which user group is impacted by this problem the most (please write 20-50 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 80-100 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem:

- **Title**: Adaptive Automation Integration

- **Explanation**: As factory automation technologies evolve, a significant problem emerges from the intersection of rapid technological advancements and varying levels of legacy systems present in manufacturing environments. Many factories are equipped with a mix of old and new machinery, where the older components lack the connectivity and data-sharing capabilities essential for today's smart manufacturing paradigms. The challenge lies in integrating advanced automation solutions, like robotics and AI-driven systems exemplified by Aniai's Alpha Grill, with these legacy systems in a cost-effective, seamless manner. This integration must not only ensure operational efficiency but also maintain or improve production quality. The problem is compounded by the diverse range of proprietary systems and protocols used by different equipment manufacturers, leading to compatibility issues. Furthermore, as consumer behavior shifts towards d

In [14]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria.
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)
        
    Be very stringent with your ratings:
        1: Does not meet expectations
        2: Meets expectations adequately
        3: Exceeds expectations significantly
    
    A rating of 3 should be reserved for exceptional performance in the given criterion.

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 50 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

**Problem: Adaptive Automation Integration**

- **Size of user group**: 3  
- **Relevancy of pain point**: 3  
- **How specific is the problem?**: 3  

**Overall score**: 9/9  

**Justification**: This issue is widespread across the factory automation sector, directly impacts operational efficiency and competitiveness in a highly relevant manner, and addresses a specific challenge of integrating new technologies with legacy systems.

---

Problem rating:

**Problem: Adaptive Energy Optimization**

- Size of user group: 3
- Relevancy of pain point: 3
- How specific is the problem: 3

**Justification**: The need for adaptive energy optimization is broad and critical across the factory automation industry, highly relevant due to increasing energy costs and sustainability goals, and specifically addresses a complex issue not solved by generic solutions.

**Overall Score**: 9/9

---

Problem rating:

**Problem**: Smart Factory Skill Gap

- **Size of user group**: 3
- **Rele

In [15]:
# third agent to give best problem 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
   Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
       - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
 
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print(completion.choices[0].message.content)
print("\n---\n")




- **Title**: Adaptive Automation Integration
- **Explanation**: In the realm of factory automation, the challenge of seamlessly integrating advanced automation technologies with existing legacy systems stands as a significant barrier. This issue is exacerbated by the diversity in equipment ages and the proprietary nature of their systems, which hinders interoperability and data sharing, essential for smart manufacturing. As factories strive to meet the increasing demand for customized products with rapid delivery schedules, the need for modular, easily upgradable automation solutions that can communicate across various platforms becomes critical. These solutions must enhance operational efficiency and production quality without substantial downtime or investment.
- **User group**: This problem predominantly impacts Factory Operations Managers.
- **Pain point**: The core pain point for operations managers lies in the struggle to enhance production efficiency and adapt to market demands 

In [16]:
problems_and_ratings = [a + b for a, b in zip(problems_list, rated_problems_list)]

def print_menu(options):
    for idx, option in enumerate(options, start=1):
        print(f"{idx}. {option}")

def get_user_choice(num_options):
    while True:
        try:
            choice = int(input("Enter your choice (number): "))
            if 1 <= choice <= num_options:
                return choice
            else:
                print(f"Please enter a number between 1 and {num_options}.")
        except ValueError:
            print("Please enter a valid integer.")

def main():
    options = problems_and_ratings
    print_menu(options)
    choice = get_user_choice(len(options))
    print(f"You chose: {options[choice - 1]}")
    return choice

if __name__ == "__main__":
    main()
    chosen_problem = problems_list[main()-1]
    print("Your choice is:\n" + chosen_problem)

1. - **Title**: Adaptive Automation Integration

- **Explanation**: As factory automation technologies evolve, a significant problem emerges from the intersection of rapid technological advancements and varying levels of legacy systems present in manufacturing environments. Many factories are equipped with a mix of old and new machinery, where the older components lack the connectivity and data-sharing capabilities essential for today's smart manufacturing paradigms. The challenge lies in integrating advanced automation solutions, like robotics and AI-driven systems exemplified by Aniai's Alpha Grill, with these legacy systems in a cost-effective, seamless manner. This integration must not only ensure operational efficiency but also maintain or improve production quality. The problem is compounded by the diverse range of proprietary systems and protocols used by different equipment manufacturers, leading to compatibility issues. Furthermore, as consumer behavior shifts towards demand f

# Getting data from Mergeflow's API

In [17]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
#vquery = 'home appliances' # topic is home appliances
# dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
# rows = str(20) # first 20 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
# url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
#headers = {
#    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
#}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


In [18]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aniai, a New York City-based robotic company specializing in kitchen solutions, has secured $12 million in Pre-Series A funding. The funding round was led by InterVest, with contributions from SV Investment, UK-based Ignite Innovation, and existing investor Capstone Partners. The company plans to utilize the funds to establish its first manufacturing facility, Factory One, in South Korea. This move aims to ensure stable production and efficient supply chain management for their flagship product, the Alpha Grill, aligning with their expansion efforts in North America and globally.

Established in 2020 and headed by CEO Gunpil Hwang, Aniai focuses on developing robotic solutions for cooking hamburgers. Their Alpha Grill stands out for its innovative approach, featuring a double-sided grill capable of cooking up to eight patties simultaneously or 200 patties per hour. By automating tasks like pressing and flipping patties, the Alpha Grill streamlines operations for restaurant brands, redu

In [19]:
Problem = chosen_problem

print(Problem)

"Title": Adaptive Energy Optimization

"Explanation": With the rapid advancements in technology and the shifting focus towards sustainability and energy efficiency, a significant problem that emerges at the intersection of these trends is the lack of adaptive energy optimization in factory automation systems. As factories become more automated, the energy consumption of these systems becomes a critical factor, not only for operational costs but also for environmental impact. Traditional automation systems are designed for peak efficiency under specific conditions, but they often lack the flexibility to adapt to changes in production demand, leading to unnecessary energy consumption during downtimes or less intensive operation periods. The integration of IoT (Internet of Things) devices, AI (Artificial Intelligence), and machine learning algorithms offers a potential solution by enabling systems that can dynamically adjust energy usage based on real-time data about the factory's operati

# Answering questions on the results with OpenAI GPT

In [20]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    The solution should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 3 sentences.
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (3 words max)
    “Solved Problem”: 1 or 2 sentence summary of the problem that is being solved 
    “Solution”: Provide a summary of the solution and how it works (in 4-5 sentences). Be dilligent on this task and make sure yo use the length limit. 
    “Impact”: How does the solution solve the specific problem (in 2-3 sentences)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution (in 4-5 sentences)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



### 1. Solution: Quantum Energy Prediction

**Summary**:  
This solution leverages quantum computing to predict factory energy needs with unprecedented accuracy and speed. By processing vast amounts of operational and environmental data in real-time, the system can forecast energy requirements for various production levels and adjust the automation systems accordingly. It utilizes machine learning algorithms optimized for quantum computing, enabling dynamic energy optimization like never before.

**Customer Perspective**:  
A factory manager inputs their production schedule, along with any potential variables such as expected downtimes or increased demand periods, into the system. The quantum computer analyzes this alongside real-time energy prices and environmental data, recommending the most energy-efficient operation modes. The system then automatically adjusts the factory's automation systems, ensuring optimal energy use without sacrificing productivity.

**Innovation Aspect**:  
W

In [21]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. When evaluating the opportunities, you should take the perspective of a potential client of these solutions, active in the specific industry of {query}.
You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Task: Evaluate the provided solutions based on detailed criteria.

Input Text: {summary_solutions}

Instructions:

Extract Solutions: Begin by identifying all solutions within the input text. For each solution, note the following details:
    - Name
    - Solved Problem
    - Solution Description
    - Impact
    - Technology Used
    - Sources
    
Develop Evaluation Criteria: Create 10 specific criteria to assess the solutions. These criteria should enable a clear comparison between the solutions, focusing on aspects such as innovation, scalability, social impact, and cost-effectiveness.

Rate Each Solution: Apply the criteria to each solution, assigning ratings on a scale from 1 to 3 (1=Lowest, 3=Highest). Be stringent with the ratings:
    1: Does not meet expectations
    2: Meets expectations adequately
    3: Exceeds expectations significantly

A rating of 3 should be reserved for exceptional performance in the given criterion.

Then, also output the total score of a solution, out of the maximum of 30 points.

Repeat for All Solutions: Follow the steps above for each solution presented in the input text.

Objective: This structured analysis will facilitate a comprehensive comparison of the solutions, highlighting their strengths and areas for improvement based on the defined criteria.

In your output, please ensure that each solution is separated by "\n---\n".
"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



### Solution 1: Quantum Energy Prediction

- **Name**: Quantum Prediction
- **Solved Problem**: Accurately forecasting energy needs in real-time for factories to optimize energy consumption without compromising productivity.
- **Solution Description**: Utilizes quantum computing and machine learning algorithms to process vast amounts of operational and environmental data, forecasting energy requirements for various production levels and adjusting automation systems for optimal energy usage.
- **Impact**: Reduces unnecessary energy consumption during downtimes or less intensive operation periods, achieving a balance between high productivity and energy efficiency.
- **Technology Used**: Quantum computing and machine learning algorithms.
- **Sources**: Inspired by "Terabase Energy" and "Numbers Station".

### Evaluation Criteria for Quantum Energy Prediction

1. **Innovation**: 3
2. **Scalability**: 2
3. **Social Impact (Environmental)**: 3
4. **Cost-Effectiveness**: 2
5. **Accuracy of P

In [22]:
# Pick the best three 

user_prompt5 = f"""
Extract and rank the solutions based on their overall ratings from the provided data: {initial_rating}.

Identify the top three solutions with the highest overall ratings out of 30 and sort them in descending order of their ratings.

For each of these top solutions, provide detailed information based on the content from: {summary_solutions}. Ensure comprehensive coverage of the points listed below:

- "Name": The solution's name, limited to a maximum of 5 words.
- "Solved Problem": A concise three-sentence summary of the problem addressed by the solution (minimum 50 words).
- "Solution": A brief summary of the solution, including its mechanism, in 4 to 5 sentences.
- "Impact": Explanation of how the solution addresses the specific problem, in 2 to 3 sentences.
- "Technology": Detailed description of the technologies used, including their application, aimed at a CTO with an engineering background for implementation purposes, in 4-5 sentences.
- "Sources": Mention any specific inspiration sources from {summary_list}, limited to 1 sentence.

Additionally, highlight the rating criteria where each solution excels.

Please separate each solution's details with "---".

"""

# Call the GPT model to generate the completion
completion = openai.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
    {"role": "system", "content": system_prompt_2},
    {"role": "user", "content": user_prompt5}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

# Correctly access the generated text from the completion response
generated_text = completion.choices[0].message.content

# Split the generated text into individual solutions based on your specified separator ("---")
solutions_list = generated_text.split("---")

# Initialize an empty dictionary to hold your final solutions
final_solutions = {}

# Populate the dictionary with distinct solutions
for index, solution_text in enumerate(solutions_list, start=1):
    if solution_text.strip():  # Ensure the string is not just whitespace
        final_solutions[index] = solution_text.strip()

# Ensure only the top 3 solutions are kept if more were accidentally generated
final_solutions = dict(list(final_solutions.items())[:3])

# 'final_solutions' now contains up to three distinct items as requested

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


### Quantum Energy Prediction
- **Name**: Quantum Prediction
- **Solved Problem**: This solution confronts the challenge of accurately forecasting energy needs in real-time for factories, aiming to optimize energy consumption without compromising productivity. Through the processing of vast amounts of data, it forecasts energy requirements for various production levels, enabling adjustments for optimal energy usage. It represents a balance between high productivity and energy efficiency.
- **Solution**: Quantum Energy Prediction leverages quantum computing and machine learning algorithms to analyze operational and environmental data, predicting energy needs with high accuracy. This system dynamically adjusts energy consumption based on real-time factory operations and energy prices, promoting efficient use of resources.
- **Impact**: By accurately predicting energy needs, the solution significantly reduces unnecessary energy consumption during downtimes, achieving a balance between mai

In [23]:
# Integrate new mergeflow data for the solutions 

# Now, based on soultions found, we need more info from Mergeflow


boolean_queries = {}

for index, solution_name in final_solutions.items():
    user_prompt6 = f"""
    
    Create a boolean query based on the Name of the solution from {solution_name}.
    
    Please use the following as an example and follow the format: 
    
    Example: for Factory Blockchain Protocol, this would be:

    "Factory Blockchain Protocol" OR "FBP" OR "blockchain for manufacturing" OR "blockchain in factories" OR "blockchain in production" OR "blockchain in supply chain" OR "blockchain in logistics" OR "blockchain in industry" OR "blockchain in industrial processes" AND factory OR manufacturing OR production OR supply chain OR logistics OR industry OR industrial processes
    

    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
        {"role": "system", "content": system_prompt_2},
        {"role": "user", "content": user_prompt6}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )

    # Extracting the content of the message from the first choice in the choices list
    query_content = completion.choices[0].message.content
    
    # Save the generated boolean query for the current solution
    boolean_queries[index] = query_content

    # Print the generated boolean query for the current solution
    print(query_content)
    print("\n----\n")
    

    

"Quantum Prediction" OR "Quantum Energy Prediction" OR "quantum computing for energy optimization" OR "quantum in factory automation" OR "quantum computing in manufacturing" OR "quantum algorithms for energy efficiency" OR "adaptive energy optimization" OR "real-time energy forecasting" AND factory OR manufacturing OR automation OR energy optimization OR productivity OR sustainability OR environmental impact OR operational costs

----

"Biomimicry Grids" OR "AI-driven Biomimicry Grids" OR "energy optimization in manufacturing" OR "sustainable energy distribution" OR "adaptive energy management" OR "AI in energy management" OR "IoT in energy optimization" OR "real-time energy optimization" OR "biomimicry in energy management" AND factory OR manufacturing OR production OR automation OR energy efficiency OR sustainability OR environmental impact

----

"Nano-grid Control" OR "Nano-grid Controllers" OR "nanotechnology in energy management" OR "AI in energy management" OR "microscopic energ

In [24]:
len(boolean_queries)

3

In [25]:
# the Mergeflow API call

dataset = '&sp=2655' # access dataset from mergeflow - in this case science publications
rows = str(3) # first 3 docs only

# Iterate through each boolean query
for solution, query in boolean_queries.items():
    # Construct the API request URL
    url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

    # Define the headers to be sent with the request
    headers = {
        'MergeflowNet-Api-Auth-Key': mergeflow_api_key
    }

    # Send the POST request
    response = requests.post(url, headers=headers)

    mergeflow_docs = []

    # Check the response status code and content
    if response.status_code == 200:
        mergeflow_response_json = response.json()
        
        # Extract and store documents information
        for document in mergeflow_response_json['Documents']:
            current_doc = {
                'title': document['Title'],
                'date': document['Date'],
                'url': document['Url'],
                'content': document['Content']
            }
            
            mergeflow_docs.append(current_doc)

    # Process or print your results here
    # For example, print titles of documents for this query
    print(f"Results for {solution}:")
    for doc in mergeflow_docs:
        print(doc['title'])
    print("\n----\n")      


Results for 1:

----

Results for 2:

----

Results for 3:

----

